## 讀檔

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 原始data
# df = pd.read_csv('C:\lab\\aigo\\30_Training Dataset_V2\\training_data.csv')
# test_data = pd.read_csv('C:\lab\\aigo\\30_Private Dataset _Private and Publict Submission Template_v2\public_private_dataset.csv')

# 已生成部分特徵的data
df = pd.read_csv('C:\lab\\aigo\\30_Training Dataset_V2\\training_data_processed.csv')
test_data = pd.read_csv('C:\lab\\aigo\\30_Private Dataset _Private and Publict Submission Template_v2\public_private_dataset_process_with_price.csv')
print(df.columns)
print(df.shape)

print(test_data.columns)
print(test_data.shape)

Index(['ID', '縣市', '鄉鎮市區', '路名', '土地面積', '使用分區', '移轉層次', '總樓層數', '主要用途',
       '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數', '橫坐標', '縱坐標', '備註',
       '主建物面積', '陽台面積', '附屬建物面積', '單價', '縣市最低單價', '縣市最高單價', '縣市平均單價',
       '縣市單價中位數', '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區平均單價', '鄉鎮市區單價中位數', '該路段最低單價',
       '該路段最高單價', '該路段平均單價', '該路段單價中位數', 'WGS84_LAT', 'WGS84_LON', '國小數量',
       '國中數量', '高中數量', '大學數量', '火車站點數量', '郵局數量', '金融機構數量', '醫院診所數量'],
      dtype='object')
(11751, 44)
Index(['ID', '縣市', '鄉鎮市區', '路名', '土地面積', '使用分區', '移轉層次', '總樓層數', '主要用途',
       '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數', '橫坐標', '縱坐標', '備註',
       '主建物面積', '陽台面積', '附屬建物面積', '縣市最低單價', '縣市最高單價', '縣市平均單價', '縣市單價中位數',
       '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區平均單價', '鄉鎮市區單價中位數', '該路段最低單價', '該路段最高單價',
       '該路段平均單價', '該路段單價中位數', 'WGS84_LAT', 'WGS84_LON', '國小數量', '國中數量', '高中數量',
       '大學數量', '火車站點數量', '郵局數量', '金融機構數量', '醫院診所數量'],
      dtype='object')
(11751, 43)


## 資料前處理

In [2]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [3]:
# 使用 LabelEncoder 將文字資料轉換為數值

def oneHotEncode(df, col_list):
    df = pd.get_dummies(df, columns=col_list,dtype=int)
    return df


def labelEncode(df, col_list):
    for col in col_list:
        df[col] = labelencoder.fit_transform(df[col])
    return df


# 處理數值特徵的偏移值
# 將負數進行平移，使其成為正數，並與正數保持平移前的相對關係
def offset_cal(df, col_list):
    for col in col_list:
        offset = abs(min(df[col]))
        df[col] = df[col] + offset
    return df


In [4]:
# 用於解決因使用dummies導致資料得column不一致的狀態，為data加上不存在的col，並補上0的值
def make_columns_consistent(data_1st, data_2nd, default_value = 0):
    # 取得train data、test data的column name
    train_columns = set(data_1st.columns)
    test_columns = set(data_2nd.columns)

    # 檢查兩個data中是否存在額外的column
    missing_columns_1st = test_columns - train_columns
    missing_columns_2nd = train_columns - test_columns

    # 將存在於data_2nd 但不存在data_1st的column加入 data_1st，並將該值設為0
    for col in missing_columns_1st:
        data_1st[col] = default_value  # 可以根据问题需要设置不同的默认值
    
    # 將存在於data_1st 但不存在data_2nd的column加入 data_2nd，並將該值設為0
    for col in missing_columns_2nd:
        data_2nd[col] = default_value  # 可以根据问题需要设置不同的默认值

    return data_1st,data_2nd

#### train data 前處理

In [5]:
# col_list = ['縣市', '主要用途', '主要建材', '建物型態']
# df = oneHotEncode(df, col_list)

# col_list = ['鄉鎮市區', '路名']
all_col = ['縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態']
df = labelEncode(df, all_col)

# 處理數值特徵的偏移值
offset_col = ['土地面積', '建物面積', '車位面積', '主建物面積', '陽台面積', '附屬建物面積']
df = offset_cal(df, offset_col)

# 將房子所除樓層進行處理
df['樓層'] = df['移轉層次']/df['總樓層數']

# 刪除ID、string的資料
df = df.drop(['ID', '縣市', '鄉鎮市區', '路名', '使用分區', '主要用途', '橫坐標', '縱坐標', '備註','醫院診所數量', '郵局數量', '金融機構數量', '縣市最低單價',
              '縣市最高單價', '縣市平均單價', '縣市單價中位數', '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區單價中位數','該路段最低單價', '該路段最高單價'], 
              axis=1)
# df = df.drop(['ID', '縣市', '使用分區', '移轉層次', '總樓層數', '建物面積', '備註', '縣市最低單價', '縣市最高單價', '縣市平均單價', '縣市單價中位數', '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區單價中位數', '該路段最低單價',
#        '該路段最高單價'], axis=1)
x = df.drop(['單價'], axis=1)
y = df['單價']


#### test data 前處理

In [6]:
# col_list = ['縣市', '主要用途', '主要建材', '建物型態']
# test_data = labelEncode(test_data, col_list)
# test_data = oneHotEncode(test_data, col_list)

# col_list = ['鄉鎮市區', '路名']
all_col = ['縣市', '鄉鎮市區', '路名', '主要用途', '主要建材', '建物型態']
test_data = labelEncode(test_data, all_col)

# 處理數值特徵的偏移值
offset_col = ['土地面積', '建物面積', '車位面積', '主建物面積', '陽台面積', '附屬建物面積']
test_data = offset_cal(test_data, offset_col)

# 將房子所除樓層進行處理
test_data['樓層'] = test_data['移轉層次']/test_data['總樓層數']

# 刪除ID、string的資料
# test_data = test_data.drop(['ID','縣市', '鄉鎮市區', '路名', '使用分區', '主要用途', '橫坐標', '縱坐標', '備註'], axis=1)
test_data = test_data.drop(['ID', '縣市', '鄉鎮市區', '路名', '使用分區', '主要用途', '橫坐標', '縱坐標', '備註','醫院診所數量', '郵局數量', '金融機構數量', '縣市最低單價', 
                            '縣市最高單價', '縣市平均單價', '縣市單價中位數', '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區單價中位數','該路段最低單價', '該路段最高單價'], 
                            axis=1)
# test_data = test_data.drop(['ID', '縣市', '使用分區', '移轉層次', '總樓層數', '建物面積', '備註', '縣市最低單價', '縣市最高單價', '縣市平均單價', '縣市單價中位數', '鄉鎮市區最低單價', '鄉鎮市區最高單價', '鄉鎮市區單價中位數', '該路段最低單價',
#        '該路段最高單價'], axis=1)
test_data.columns

Index(['土地面積', '移轉層次', '總樓層數', '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數',
       '主建物面積', '陽台面積', '附屬建物面積', '鄉鎮市區平均單價', '該路段平均單價', '該路段單價中位數',
       'WGS84_LAT', 'WGS84_LON', '國小數量', '國中數量', '高中數量', '大學數量', '火車站點數量',
       '樓層'],
      dtype='object')

In [7]:
print("x.shape: ",x.shape)
print("test_data.shape: ",test_data.shape)

x.shape:  (11751, 23)
test_data.shape:  (11751, 23)


In [8]:
print(x.columns)
print(test_data.columns)

Index(['土地面積', '移轉層次', '總樓層數', '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數',
       '主建物面積', '陽台面積', '附屬建物面積', '鄉鎮市區平均單價', '該路段平均單價', '該路段單價中位數',
       'WGS84_LAT', 'WGS84_LON', '國小數量', '國中數量', '高中數量', '大學數量', '火車站點數量',
       '樓層'],
      dtype='object')
Index(['土地面積', '移轉層次', '總樓層數', '主要建材', '建物型態', '屋齡', '建物面積', '車位面積', '車位個數',
       '主建物面積', '陽台面積', '附屬建物面積', '鄉鎮市區平均單價', '該路段平均單價', '該路段單價中位數',
       'WGS84_LAT', 'WGS84_LON', '國小數量', '國中數量', '高中數量', '大學數量', '火車站點數量',
       '樓層'],
      dtype='object')


#### 解決因使用dummies導致資料得column不一致的狀態

In [9]:
# 解決因使用dummies導致資料得column不一致的狀態
x, test_data = make_columns_consistent(x, test_data)

In [10]:
print("x.shape: ",x.shape)
print("x.shape: ",x.shape[1])
print("test_data.shape: ",test_data.shape)

x.shape:  (11751, 23)
x.shape:  23
test_data.shape:  (11751, 23)


In [ ]:
x.columns


In [ ]:
len(x.columns)

In [ ]:
test_data.columns

In [ ]:
# # 找出包含小于 0 的值的行和列
# rows, cols = np.where(x < 0)

# # 打印包含小于 0 的值的行和列
# for row, col in zip(rows, cols):
#     print(f"Row {row}, Column {df.columns[col]}: {df.iloc[row, col]}")

## 特徵篩選

In [11]:
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.feature_selection import chi2  # 选择合适的评分函数，如卡方检验

In [12]:
# SelectKBest 實例，使用 f_regression 評分函數
selector = SelectKBest(score_func=f_regression, k=15)  # 選擇最重要的 K 個特徵

selector.fit_transform(x, y)

selected_features = selector.get_support()

In [13]:
print("x shape:", x.shape)
print("selected_features shape:", selected_features.shape)

x shape: (11751, 23)
selected_features shape: (23,)


In [ ]:
# selected_features

In [ ]:
# x.columns[selected_features]

In [ ]:
# x = x[x.columns[selected_features]]
# x

In [ ]:
# test_data = test_data[test_data.columns[selected_features]]
# test_data

## 模型訓練

### import package、自定義dataset、定義模型

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

from xgboost import XGBClassifier
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score


class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]


# 自定義 MAPE 評估函數
def mape_eval(y_pred, data):
    y_true = data.get_label()
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return 'MAPE', mape, False



#### 模型

### data進行transform及切割

In [15]:
# 標準化特徵
scaler = StandardScaler()
X_train = scaler.fit_transform(x)

# 將數據劃分為訓練集和測試集
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=42)

In [16]:
# print('X_train ',type(X_train))
# print('X_val ',type(X_val))

# print('y_train ',type(y_train))
# print('y_val ',type(y_val))


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


### 查看是否有gpu、定義loss及optimizer

In [18]:
# 检查是否有可用的GPU，如果有，选择第一个可用的GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

ExtraTreesRegressor(n_estimators=300, random_state=42)

## XGBRegressor

In [ ]:
# XGBRegressor
# model = XGBRegressor(learning_rate=0.1,)
# model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric=['rmse', 'mae'], early_stopping_rounds=10) 

## LightGBM

In [ ]:
# LightGBM
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)


# 创建 LightGBM 模型并使用自定义评估函数
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

params = {
    'objective': 'regression',
    'metric': 'mae',    # MAE仅用于监控训练过程，而不是最终评估指标
    # 其他参数
} 
model = lgb.train(params, train_data, valid_sets=[valid_data], feval=mape_eval, early_stopping_rounds=10, num_boost_round=1000)

## RandomForestRegressor

In [ ]:
# RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=42)  # 可根据需要调整超参数
model.fit(X_train, y_train)


##　ExtraTreesRegressor

In [ ]:
#　ExtraTreesRegressor
model = ExtraTreesRegressor(n_estimators=300, random_state=42)
model.fit(X_train, y_train)

In [19]:
y_pred = model.predict(X_val)

In [20]:
mse = mean_squared_error(y_val, y_pred)
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)
mape = mean_absolute_percentage_error(y_val, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)
print("Mean Absolute Percentage Error (MAPE):", mape)


Mean Squared Error: 0.10868655487413895
Mean Absolute Error: 0.15025042854381263
R-squared: 0.8911702213531817
Mean Absolute Percentage Error (MAPE): 7.672704195666823


## 使用欲預測的data進行test

### 這裡使用public data

In [21]:
# 標準化特徵
scaler = StandardScaler()
X_test = scaler.fit_transform(test_data)

X_test_tensor = torch.Tensor(X_test)

In [22]:
# 使用模型進行預測
y_pred = model.predict(X_test)

# test_predictions = test_outputs.cpu().detach().numpy()
y_pred

array([1.6070133 , 1.69264671, 2.732037  , ..., 1.6142353 , 1.18621662,
       2.0275133 ])

In [23]:
# 读取原始CSV文件
result = pd.read_csv('C:\lab\\aigo\\30_Private Dataset _Private and Publict Submission Template_v2\public_private_submission_template.csv')

# 将NumPy数组的数据覆盖到第二列
result['predicted_price'] = y_pred

# 保存DataFrame回到CSV文件
result.to_csv('C:\lab\\aigo\\30_Private Dataset _Private and Publict Submission Template_v2\submit\extratree_estimators_100_1108_mape7.672704.csv', index=False)

## 查看特徵對模型的影響力

In [ ]:
feature_importance = model.feature_importance(importance_type='split')  # 或 'gain'
#
#  获取特征名称
feature_names = x.columns

In [ ]:
feature_importance_ranking = sorted(
    zip(feature_names, feature_importance),
    key=lambda x: x[1],
    reverse=True
)
feature_importance_ranking